This is a simple example project where I want to extract parameters from piece of spectrum data that I have. My gaussian function has the following form:

$f(x) = a \mathrm{e}{\frac{-(x-c)^2}{2c^2}} + d$

Where $a$ is a normalisation coefficient, $b$ is the center point, $c$ defines with the width of the curve and $d$ is the height above the x axis.

First lets load and plot the data

In [10]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from bokeh.plotting import figure, output_notebook, show
output_notebook()
import numpy as np

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['figure', 'show']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Loading BokehJS ...

In [11]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
x = np.linspace(970, 1050, 500)
y = gaussian(x, 995, 1)*12000 + np.random.random_sample(size=x.shape) * 1500
np.savetxt('./data/data.txt', np.vstack((x, y)).T)

In [21]:
x, y = np.loadtxt('./data/data.txt').T

p = figure(title="data plot", x_axis_label='wavelength [nm]', y_axis_label='counts [a.u.]')
p.line(x, y, legend="Temp.", line_width=2)
show(p)

Clearly the data is a nice gaussian, so lets fit the function to get the center point and 
full-width at half max, which is given by 

$FWHM = 2\sqrt{2 ln(2)}$

First we import some code which contains the gauss equation, and the optimize function from scipy to do the curve fitting.

In [15]:
from scipy.optimize import curve_fit

def gauss(t, a, b, c, d):
    return a*np.exp(-((t-b)**2)/(2*c**2)) + d

In [16]:
p0 = [700, 990, 2, 0]
params, cov = curve_fit(gauss, x, y, p0=p0)
err = np.sqrt(np.diag(cov))

print("a = %lf +- %lf" % (params[0], err[0]))
print("b = %lf +- %lf" % (params[1], err[1]))
print("c = %lf +- %lf" % (params[2], err[2]))
print("d = %lf +- %lf" % (params[3], err[3]))

a = 12032.075742 +- 164.192818
b = 995.017541 +- 0.015373
c = 0.979393 +- 0.015550
d = 748.226468 +- 20.324062


In [20]:
p = figure(title="fit overlay", x_axis_label='wavelength [nm]', y_axis_label='counts [a.u.]')
p.line(x, y, legend="Temp.", line_width=1)
p.line(x, gauss(x, *params), line_width=2, line_color='red')
show(p)

So here we have a pretty good fit, and we were able to find the intensity, center point and width of the curve.
Lets just finally calculate the FWHM

In [7]:
FWHM = 2*np.sqrt(2*np.log(2))
print(FWHM)

2.35482004503
